# Imports

In [ ]:
from trièst import Trièst

'''
dataset links:
 - twitch-gamers -> https://snap.stanford.edu/data/twitch_gamers.html
'''

graph_name = 'large_twitch'
triest = Trièst(graph_name)

# Trièst algorithm

In [ ]:
M = 1000000
T = 10000
triest.trièst_base(M, T)
print(f'Estimated number of triangles: {triest.t_global}')


# Checking the results using Networkx

In [ ]:
triangles = triest.triangles_count(10000)
print(f'Number of triangles in the graph: {sum(triangles) // 3}') # type: ignores